In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("mixtools")
BiocManager::install("bcellViper")
BiocManager::install("viper")

package 'BiocManager' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jmjun\AppData\Local\Temp\RtmpqmQU35\downloaded_packages


Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'BiocVersion', 'mixtools'
also installing the dependencies 'lattice', 'Matrix', 'kernlab', 'MASS', 'segmented', 'survival'



package 'lattice' successfully unpacked and MD5 sums checked
package 'Matrix' successfully unpacked and MD5 sums checked
package 'kernlab' successfully unpacked and MD5 sums checked
package 'MASS' successfully unpacked and MD5 sums checked
package 'segmented' successfully unpacked and MD5 sums checked
package 'survival' successfully unpacked and MD5 sums checked
package 'BiocVersion' successfully unpacked and MD5 sums checked
package 'mixtools' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jmjun\AppData\Local\Temp\RtmpqmQU35\downloaded_packages


Old packages: 'digest', 'evaluate', 'htmltools', 'IRkernel', 'jsonlite',
  'Rcpp', 'repr', 'uuid'
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'bcellViper'
also installing the dependencies 'BiocGenerics', 'Biobase'



package 'BiocGenerics' successfully unpacked and MD5 sums checked
package 'Biobase' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jmjun\AppData\Local\Temp\RtmpqmQU35\downloaded_packages


installing the source package 'bcellViper'

Old packages: 'digest', 'evaluate', 'htmltools', 'IRkernel', 'jsonlite',
  'Rcpp', 'repr', 'uuid'
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'viper'
also installing the dependencies 'class', 'e1071', 'KernSmooth'



package 'class' successfully unpacked and MD5 sums checked
package 'e1071' successfully unpacked and MD5 sums checked
package 'KernSmooth' successfully unpacked and MD5 sums checked
package 'viper' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jmjun\AppData\Local\Temp\RtmpqmQU35\downloaded_packages


Old packages: 'digest', 'evaluate', 'htmltools', 'IRkernel', 'jsonlite',
  'Rcpp', 'repr', 'uuid'


In [2]:
library(viper)
library(bcellViper)
library(mixtools)

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


## creating regulon object

In [5]:
createRegulon = function(file_name){ 
  viper_regulon = list()
  # Load network and format regulons
  net = read.delim(file_name, stringsAsFactors = F)
  
  # Check number of TFs
  tfs = unique(net$TF)
  message('Interactions: ', nrow(net))
  message('Genes: ', length(unique(net$target)))
  message('TFs: ', length(tfs))

  #  Iterate each TF to build the reguon object and ...
  for (tf in tfs ){
    # Define targets
    tf_targets = subset(net, TF == tf)$target
      
    # Define TF mode
    tf_targets_sign = subset(net, TF == tf)$mode
    
    # Define regulation weight
    tf_targets_weight = rep(1, length(tf_targets))
      
    # Build VIPER regulon
    viper_regulon[[tf]]$tfmode = tf_targets_sign # NOTE: tfmode has to be the first element of the list. Otherwise msviper will fail.
    names(viper_regulon[[tf]]$tfmode) = tf_targets
    viper_regulon[[tf]]$likelihood = tf_targets_weight
  }
  return(viper_regulon)
}
trrust_regulon=createRegulon(file_name='data/TRRUST2_TF/trrust2_TF_target_processed.txt')

Interactions: 4800
Genes: 1568
TFs: 627


## run msviper

In [6]:
msviper_func = function(fc_file){ 
    fc.df = read.delim(fc_file, stringsAsFactors = F, sep='\t')
    # dataframe to matrix
    fc.mx=as.matrix(fc.df)
    # get effect column
    effect=fc.mx[,ncol(fc.mx)]
    # remove effect column & transpose
    fc.mx=fc.mx[,-ncol(fc.mx)]
    fc.mx=t(fc.mx)
    
    # run msviper
    res=msviper(fc.mx, trrust_regulon, minsize=1,verbose = T)
    
    # get TF activity
    TFA=res$es$nes.bt
    TFA=t(TFA)
    
    # append 
    TFA=cbind(TFA,effect)
    message('## TFA matrix + effect')
    message('nrow(exp): ', nrow(TFA))
    message('ncol(TF): ', ncol(TFA))
    print(TFA[1:5,(ncol(TFA)-5):ncol(TFA)])
    cell=strsplit(fc_file,'_')[[1]][4]
    write.table(TFA, file=paste('result/TF_activity_viper/TFA',cell,'v_raw.txt',sep='_'), sep='\t',row.names=F, col.names=T)
}
msviper_func('result/foldchange_with_effect/FC_A375_effect.txt')
msviper_func('result/foldchange_with_effect/FC_HT29_effect.txt')

Computing regulon enrichment with aREA algorithm


  |======================================================================| 100%

## TFA matrix + effect
nrow(exp): 456
ncol(TF): 628


         ZNF382     ZNF383     ZNF444      ZNF76       ZNRD1 effect
[1,] -0.8254295  1.0023214 -1.2623528 -0.7974027 -1.39298216      0
[2,] -0.4969884 -0.8043002  0.1849407  1.3995490 -1.06589548      0
[3,] -1.5239552  0.3991710 -1.1959217 -0.1366888  0.06466780      0
[4,]  0.6262098 -1.6627000  0.4541759 -0.4955398 -0.01170077      0
[5,] -0.1539613 -1.9344403  0.1697754 -0.7875495 -1.02012409      0


Computing regulon enrichment with aREA algorithm


  |======================================================================| 100%

## TFA matrix + effect
nrow(exp): 456
ncol(TF): 628


         ZNF382     ZNF383     ZNF444      ZNF76       ZNRD1 effect
[1,] -0.7772658 0.96489652 -1.2328004  0.5991745  0.38742988      0
[2,] -1.9543563 0.92877660 -1.3482273  0.2455776 -0.05823963      0
[3,] -0.8234642 0.09173511 -0.1272346  1.0840203  1.97241875      0
[4,] -0.6724340 1.13956709 -1.4850888  0.9369768  1.12331787      0
[5,] -1.6080418 0.21123785 -1.0090724 -1.1013925  1.07594782      0
